In [ ]:
# ===========================================================================
# DIANI Mamoudou Sékou
#
# Avril 2022
#===========================================================================

In [ ]:
# Importation des bibliothèques nécessaires
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Input, BatchNormalization, UpSampling2D, Activation, Dropout, Flatten, Dense
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import backend as K

# Configuration des GPUs et CPUs
config = tf.compat.v1.ConfigProto(device_count={'GPU': 2, 'CPU': 4})
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess);

In [ ]:
# Declaration des chemins vers les fichiers cibles

# Le dossier principal qui contient les données
mainDataPath = "vache_elephant/"

# Le dossier contenant les images d'entrainement
trainPath = mainDataPath + "entrainement"

# Le dossier contenant les images de validation
validationPath = mainDataPath + "validation"

# Le nom du fichier du modèle à sauvegarder
model_path = "Model.hdf5"

In [ ]:
# Declaration des variable 

# Le nombre d'images d'entrainement
training_ds_size = 1992  
validation_ds_size = 408  


# Configuration des  images
image_scale = 96 
image_channels = 3 
images_color_mode = "rgb" 
image_shape = (image_scale, image_scale,
               image_channels)  

# Configuration des paramètres d'entrainement
fit_batch_size = 70  
fit_epochs = 100  

In [ ]:
# ==========================================
# CHARGEMENT DES IMAGES===========
# ==========================================

# training_data_generator: charge les données d'entrainement
# Les images sont normalisées (leurs pixels divisées par 255)
training_data_generator = ImageDataGenerator(rescale=1. / 255, validation_split=0.17)

# validation_data_generator: charge les données de validation 
# Les images sont normalisées (leurs pixels divisées par 255)
validation_data_generator = ImageDataGenerator(rescale=1. / 255)

# training_generator: indique la méthode de chargement des données d'entrainement
training_generator = training_data_generator.flow_from_directory(
    trainPath, 
    color_mode =images_color_mode, 
    target_size=(image_scale, image_scale),
    batch_size = training_ds_size, 
    class_mode ="input", 
    subset='training') 

# validation_generatory: indique la méthode de chargement des données de validation
validation_generator = training_data_generator.flow_from_directory(
    trainPath, 
    color_mode =images_color_mode, 
    target_size=(image_scale, image_scale),
    batch_size = validation_ds_size, 
    class_mode ="input", 
    subset='validation') 

# On charge les données d'entrainement 
(x_train, _) = training_generator.next()
# On charge les données de validation 
(x_val, _) = validation_generator.next()

In [ ]:
# Couche d'entrée
input_layer = Input(shape=image_shape)

In [ ]:


# Encodage
def encoder(input):
    x = Conv2D(512, (3, 3), padding='same')(input)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(256, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    
    x = Conv2D(128, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    return encoded


# Partie de décodage (qui reconstruit les images à partir de leur embedding ou la sortie de l'encodeur)
def decoder(encoded):
    x = Conv2D(128, (3, 3), padding='same')(encoded)
    x = Activation('relu')(x)
    x = UpSampling2D((2, 2))(x)

    x = Conv2D(256, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = UpSampling2D((2, 2))(x)

    x = Conv2D(512, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(image_channels, (3, 3), padding='same')(x)
    decoded = Activation('relu')(x)
    return decoded


# Déclaration du modèle
model = Model(input_layer, decoder(encoder(input_layer)))

# Affichage des paramétres du modèle
model.summary()

# Compilation du modèle :
model.compile(loss='mse', optimizer='adam', metrics=['mse'])

In [ ]:

# Savegarder le modèle avec le minimum loss sur les données de validation (monitor='val_loss')
modelcheckpoint = ModelCheckpoint(filepath=model_path,
                                  monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

# entrainement du modèle
autoencoder = model.fit(x_train, x_train,
                       epochs=fit_epochs, # nombre d'epochs
                       batch_size=fit_batch_size, # nombre d'images entrainées ensemble
                       verbose=1, # mets cette valeur à 0, si vous voulez ne pas afficher les détails d'entrainement
                       callbacks=[modelcheckpoint], # les fonctions à appeler à la fin de chaque epoch (dans ce cas modelcheckpoint: qui sauvegarde le modèle)
                       shuffle=False, # On ne boulverse pas les données
                       validation_data=(x_val, x_val)) # données de validation


plt.plot(autoencoder.history['loss'])
plt.plot(autoencoder.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
fig = plt.gcf()
plt.show()

